In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

from multiprocess import Pool

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from visualization_utilities import *
from registration_utilities import *
from annotation_utilities import *

Setting environment for AWS compute node


No vtk


# Compare different classifiers

In [2]:
stack = 'MD642'

In [3]:
setting1 = 2
setting2 = 10
# descriptive1 = 'Nissl-trained'
# descriptive2 = 'NTB-trained'

In [4]:
first_sec, last_sec = metadata_cache['section_limits'][stack]

In [5]:
df, _ = DataManager.load_annotation_v3(stack=stack)
sections_containing_structure = {structure: set(df[df['name']==structure]['section'].values) for structure in all_known_structures}

'No object named structures in the file'


Annotation has no structures.


In [12]:
for structure in all_known_structures:
    
    print structure
    
    if structure in sections_containing_structure:
        sections = sections_containing_structure[structure]
    else:
#         sections = range(first_sec, last_sec + 1)
        continue
    
    for sec in sections:
        fn = metadata_cache['sections_to_filenames'][stack][sec]
        
        try:
            viz1 = imread(DataManager.get_scoremap_viz_filepath(stack=stack, section=sec, structure=structure, setting=setting1))
            viz2 = imread(DataManager.get_scoremap_viz_filepath(stack=stack, section=sec, structure=structure, setting=setting2))

            cv2.putText(viz1, '%d %s %d' % (sec, structure, setting1), (50, 50), cv2.FONT_HERSHEY_DUPLEX, 2, ((0,0,0)), 3)
            cv2.putText(viz2, '%d %s %d' % (sec, structure, setting2), (50, 50), cv2.FONT_HERSHEY_DUPLEX, 2, ((0,0,0)), 3)

            viz = np.hstack([viz1, viz2])

            viz_fp = os.path.join('/home/yuncong/csd395/CSHL_scoremap_viz_juxtaposed', structure, stack, fn + '_setting_%d_vs_setting_%d.jpg' % (setting1, setting2))
            create_if_not_exists(os.path.dirname(viz_fp))

            imsave(viz_fp, viz)
        except:
            continue

5N
6N
7N
7n
Amb
LC
LRt
Pn
Tz
VLL
RMC
SNC
SNR
3N
4N
Sp5I
Sp5O
Sp5C
PBG
10N
VCA
VCP
DC
AP
12N
RtTg
sp5
outerContour
SC
IC


In [ ]:
for sec in range(first_sec, last_sec + 1):
    
    fn = metadata_cache['sections_to_filenames'][stack][sec]
    if is_invalid(fn):
        continue
        
    for structure in all_known_structures:
        
        try:
            viz1 = imread(DataManager.get_scoremap_viz_filepath(stack=stack, section=sec, structure=structure, setting=setting1))
            viz2 = imread(DataManager.get_scoremap_viz_filepath(stack=stack, section=sec, structure=structure, setting=setting2))

            cv2.putText(viz1, '%d %s %d' % (sec, structure, setting1), (50, 50), cv2.FONT_HERSHEY_DUPLEX, 2, ((0,0,0)), 3)
            cv2.putText(viz2, '%d %s %d' % (sec, structure, setting2), (50, 50), cv2.FONT_HERSHEY_DUPLEX, 2, ((0,0,0)), 3)
            
            viz = np.hstack([viz1, viz2])

            viz_fp = os.path.join('/home/yuncong/csd395/CSHL_scoremap_viz_juxtaposed', structure, stack, fn + '_setting_%d_vs_setting_%d.jpg' % (setting1, setting2))
            create_if_not_exists(os.path.dirname(viz_fp))

            imsave(viz_fp, viz)
        except:
            continue

# Compare Different Warpings

In [11]:
stack_moving = 'atlasV5'
# stack_fixed = 'MD652'
stacks_fixed = ['MD653', 'MD657', 'MD658']

In [12]:
detector_id = 13
warp_settings = [16, 17]

In [13]:
for stack_fixed in stacks_fixed:
    basenames = [DataManager.get_warped_volume_basename(stack_m=stack_moving,
                                                        stack_f=stack_fixed,
                                                       detector_id_f=detector_id,
                                                        warp_setting=warp) for warp in warp_settings]
    sidebyside_basename = '_sideByside_'.join(basenames)
    
    valid_sections = metadata_cache['valid_sections'][stack_fixed]
    
    def generate_juxtaposed_viz(sec, warp_settings, sidebyside_basename):

        vizs = []
        for warp in warp_settings:
            viz_fp = DataManager.get_alignment_viz_filepath(stack_m=stack_moving,
                                                    stack_f=stack_fixed,
                                                    detector_id_f=detector_id,
                                                            prep_id_f=2,
                                                    warp_setting=warp,
                                              section=sec, out_downscale=8)
            download_from_s3(viz_fp)
            viz = imread(viz_fp)
            vizs.append(viz)

        viz = np.hstack(vizs)

        viz_fp = os.path.join('/shared/CSHL_registration_viz_juxtaposed', 
                              sidebyside_basename,
                              sidebyside_basename + '_%04d.jpg' % sec)

        create_parent_dir_if_not_exists(viz_fp)
        imsave(viz_fp, viz)
        upload_to_s3(viz_fp)

        return viz
    
    t = time.time()

    pool = Pool(4)
    pool.map(lambda sec: generate_juxtaposed_viz(sec, warp_settings, sidebyside_basename), valid_sections)
    pool.close()
    pool.join()

    sys.stderr.write("Time: %.2f seconds.\n" % (time.time() - t)) # 178s

aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume_0086.jpg" "s3://mousebrainatlas-data/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume_0086.jpg"
aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_sco

aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume_0107.jpg" "s3://mousebrainatlas-data/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume_0107.jpg"
aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_sco

aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume_0092.jpg" "s3://mousebrainatlas-data/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume_0092.jpg"
aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_sco

aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume_0095.jpg" "s3://mousebrainatlas-data/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume_0095.jpg"
aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_sco

aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume_0098.jpg" "s3://mousebrainatlas-data/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume_0098.jpg"
aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD653_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD653_detector13_down32_sco

...

aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume_0349.jpg" "s3://mousebrainatlas-data/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume_0349.jpg"
aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_sco

aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume_0353.jpg" "s3://mousebrainatlas-data/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume_0353.jpg"
aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_sco

aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume_0342.jpg" "s3://mousebrainatlas-data/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume_0342.jpg"
aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_sco

aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume_0345.jpg" "s3://mousebrainatlas-data/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume_0345.jpg"
aws s3 cp "/shared/CSHL_registration_viz_juxtaposed/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_scoreVolume_sideByside_atlasV5_down32_scoreVolume_warp17_MD658_detector13_down32_scoreVolume/atlasV5_down32_scoreVolume_warp16_MD658_detector13_down32_sco

Time: 178.09 seconds.
